In [1]:
!pip install numpy

In [2]:
import numpy as np

## Reading and processing text
with open('1268-0.txt', 'r', encoding="utf8") as fp:
    text=fp.read()
    
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')

text = text[start_indx:end_indx]
char_set = set(text)
print('Total Length:', len(text))
print('Unique Characters:', len(char_set))

Total Length: 1112350
Unique Characters: 80


In [3]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text],
    dtype=np.int32)

print('Text encoded shape: ', text_encoded.shape)

print(text[:15], '     == Encoding ==> ', text_encoded[:15])
print(text_encoded[15:21], ' == Reverse  ==> ', ''.join(char_array[text_encoded[15:21]]))

Text encoded shape:  (1112350,)
THE MYSTERIOUS       == Encoding ==>  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]  == Reverse  ==>  ISLAND


In [4]:
for ex in text_encoded[:5]:
    print('{} -> {}'.format(ex, char_array[ex]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [5]:
seq_length = 40
chunk_size = seq_length + 1

text_chunks = [text_encoded[i:i+chunk_size] 
               for i in range(len(text_encoded)-chunk_size+1)] 

## inspection:
for seq in text_chunks[:1]:
    input_seq = seq[:seq_length]
    target = seq[seq_length] 
    print(input_seq, ' -> ', target)
    print(repr(''.join(char_array[input_seq])), 
          ' -> ', repr(''.join(char_array[target])))

[44 32 29  1 37 48 43 44 29 42 33 39 45 43  1 33 43 36 25 38 28  1  6  6
  6  0  0  0  0  0 40 67 64 53 70 52 54 53  1 51]  ->  74
'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'  ->  'y'


In [6]:
!pip install torch

In [7]:
import torch

from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, text_chunks):
        self.text_chunks = text_chunks

    def __len__(self):
        return len(self.text_chunks)
    
    def __getitem__(self, idx):
        text_chunk = self.text_chunks[idx]
        return text_chunk[:-1].long(), text_chunk[1:].long()
    
seq_dataset = TextDataset(torch.tensor(text_chunks, dtype=torch.long))


/var/folders/pn/m2nfshws0wd37gvf2z0f6v040000gn/T/ipykernel_17748/254275106.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  seq_dataset = TextDataset(torch.tensor(text_chunks, dtype=torch.long))


In [9]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    print("Using MPS device")
else:
    print("MPS (Metal Performance Shaders) not available")

Using MPS device


In [10]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Using device:", device)


Using device: mps


In [11]:
for i, (seq, target) in enumerate(seq_dataset):
    print(' Input (x):', repr(''.join(char_array[seq])))
    print('Target (y):', repr(''.join(char_array[target])))
    print()
    if i == 1:
        break
    

 Input (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
Target (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

 Input (x): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'
Target (y): 'E MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by '



In [12]:
from torch.utils.data import DataLoader
 
batch_size = 32

torch.manual_seed(1)
seq_dl = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


## LSTM

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn_hidden_size = rnn_hidden_size
        self.lstm = nn.LSTM(embed_dim, rnn_hidden_size, batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size, vocab_size)

    def forward(self, x, hidden, cell):
        out = self.embedding(x)
        out, (hidden, cell) = self.lstm(out, (hidden, cell))
        out = self.fc(out)
        return out, hidden, cell

    def init_hidden(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.rnn_hidden_size).to(device)
        cell = torch.zeros(1, batch_size, self.rnn_hidden_size).to(device)
        return hidden, cell

## RNN 

In [14]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = nn.RNN(embed_dim, rnn_hidden_size, batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size, vocab_size)

    def forward(self, x, hidden):
        out = self.embedding(x)
        out, hidden = self.rnn(out, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.rnn_hidden_size).to(device)
        return hidden


## 2RNN

In [15]:
import torch
import torch.nn as nn

class TwoLayerRNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn_hidden_size = rnn_hidden_size
        #  Add num_layers=2 for 2 RNN layers
        self.rnn = nn.RNN(embed_dim, rnn_hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size, vocab_size)

    def forward(self, x, hidden):
        out = self.embedding(x)
        out, hidden = self.rnn(out, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        #  Now hidden state needs 2 layers
        hidden = torch.zeros(2, batch_size, self.rnn_hidden_size).to(device)
        return hidden


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

vocab_size = len(char_array)
embed_dim = 256
rnn_hidden_size = 256

lstm_model = LSTMModel(vocab_size, embed_dim, rnn_hidden_size).to(device)
rnn_model = RNNModel(vocab_size, embed_dim, rnn_hidden_size).to(device)
two_rnn_model = TwoLayerRNNModel(vocab_size, embed_dim, rnn_hidden_size).to(device)

# Save models
torch.save(lstm_model.state_dict(), 'lstm_model.pth')
torch.save(rnn_model.state_dict(), 'rnn_model.pth')
torch.save(two_rnn_model.state_dict(), 'two_rnn_model.pth')

print("✅ Models saved!")


Using device: cpu
✅ Models saved!


In [17]:
# --- Training Function LSTM AND RNN ---
def train_model(model, model_type='lstm', num_epochs=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch in seq_dl:  # YOUR DataLoader here
            input_seq, target_seq = batch
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)

            batch_size = input_seq.size(0)

            optimizer.zero_grad()

            if model_type == 'lstm':
                hidden, cell = model.init_hidden(batch_size)
                outputs, hidden, cell = model(input_seq, hidden, cell)
            else:  # RNN
                hidden = model.init_hidden(batch_size)
                outputs, hidden = model(input_seq, hidden)

            # Reshape outputs and targets to match CrossEntropyLoss expectation
            outputs = outputs.reshape(-1, vocab_size)
            target_seq = target_seq.reshape(-1)

            loss = criterion(outputs, target_seq)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == target_seq).sum().item()
            total += target_seq.numel()

        accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy*100:.2f}%')

In [18]:
# --- Training Function for TwoLayerRNNModel ---
def train_two_rnn_model(model, num_epochs=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for batch in seq_dl:  # YOUR DataLoader here
            input_seq, target_seq = batch
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)

            batch_size = input_seq.size(0)

            optimizer.zero_grad()

            # 🌟 For TwoLayerRNNModel
            hidden = model.init_hidden(batch_size)
            outputs, hidden = model(input_seq, hidden)

            # Reshape outputs and targets to match CrossEntropyLoss expectation
            outputs = outputs.reshape(-1, vocab_size)
            target_seq = target_seq.reshape(-1)

            loss = criterion(outputs, target_seq)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == target_seq).sum().item()
            total += target_seq.numel()

        accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy*100:.2f}%')


## Train LSTM and RNN

In [ ]:
# --- Load and Train LSTM Model ---
print("\n🔵 Training LSTM Model")
lstm_loaded = LSTMModel(vocab_size, embed_dim, rnn_hidden_size).to(device)
lstm_loaded.load_state_dict(torch.load('lstm_model.pth'))

train_model(lstm_loaded, model_type='lstm')

# --- Load and Train RNN Model ---
print("\n🟢 Training RNN Model")
rnn_loaded = RNNModel(vocab_size, embed_dim, rnn_hidden_size).to(device)
rnn_loaded.load_state_dict(torch.load('rnn_model.pth'))

train_model(rnn_loaded, model_type='rnn')

## Train 2RNN

In [34]:
# --- Load and Train TwoLayer RNN Model ---
print("\n🟣 Training TwoLayer RNN Model")
two_rnn_loaded = TwoLayerRNNModel(vocab_size, embed_dim, rnn_hidden_size).to(device)
two_rnn_loaded.load_state_dict(torch.load('two_rnn_model.pth'))

train_two_rnn_model(two_rnn_loaded)


🟣 Training TwoLayer RNN Model


C:\Users\brapo\AppData\Local\Temp\ipykernel_5200\3631671914.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  two_rnn_loaded.load_state_dict(torch.load('two_rnn_model.pth'

Epoch [1/3], Loss: 41971.4966, Accuracy: 62.89%
Epoch [2/3], Loss: 39707.5141, Accuracy: 64.54%
Epoch [3/3], Loss: 39771.8634, Accuracy: 64.46%


In [38]:
def sample(model, starting_str, len_generated_text=500, scale_factor=1.0):
    encoded_input = torch.tensor([char2int[s] for s in starting_str]).to(device)
    encoded_input = torch.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    # Check if model is LSTM (has 2 hidden states) or RNN (has 1 hidden state)
    hidden = model.init_hidden(1)
    if isinstance(hidden, tuple):
        hidden, cell = hidden
    else:
        cell = None  # no cell for RNN

    hidden = hidden.to(device)
    if cell is not None:
        cell = cell.to(device)

    for c in range(len(starting_str) - 1):
        if cell is not None:
            _, hidden, cell = model(encoded_input[:, c].view(1, 1), hidden, cell)
        else:
            _, hidden = model(encoded_input[:, c].view(1, 1), hidden)

    last_char = encoded_input[:, -1]
    for i in range(len_generated_text):
        if cell is not None:
            logits, hidden, cell = model(last_char.view(1, 1), hidden, cell)
        else:
            logits, hidden = model(last_char.view(1, 1), hidden)

        logits = torch.squeeze(logits, 0)
        scaled_logits = logits * scale_factor
        m = torch.distributions.Categorical(logits=scaled_logits)
        last_char = m.sample()

        generated_str += str(char_array[last_char])

    return generated_str



In [36]:
# Move models to evaluation mode
lstm_loaded.eval()
rnn_loaded.eval()
two_rnn_loaded.eval()

# Define how many samples you want
num_samples = 2

print("\n🔵 LSTM Generated Texts:")
for i in range(num_samples):
    print(f"\nSample {i+1}:")
    print(sample(lstm_loaded, starting_str='The island'))

print("\n🟢 RNN Generated Texts:")
for i in range(num_samples):
    print(f"\nSample {i+1}:")
    print(sample(rnn_loaded, starting_str='The island'))

print("\n🟣 Two-Layer RNN Generated Texts:")
for i in range(num_samples):
    print(f"\nSample {i+1}:")
    print(sample(two_rnn_loaded, starting_str='The island'))



🔵 LSTM Generated Texts:

Sample 1:
The island! I should know the twild it markles.”

River,
which belonged to the world, when a very nine discoveries at the top of this act again. While Neb,” and evidently found lay to be followed. Pencroft soon as
the height of smoke against eleven days. A finall side, phoys alone in the first were managed to return to see at the foxes, fell less proportion Britgle will have to prevent a chance of a slight of it frost.”

“We are unhairs. They must
find him from extinction. He gave the
passage, carried the

Sample 2:
The island in breadth, and youn leave any wise and
devoiced that he would have a greater, a fiber or even a new which I have no hour, he desired, the sailor’s sapiliting project, when Captain Nemo’s arms of
little rescaping by the hunt, found themethout the castaways could not be tons of
his idea. “At sufficient shore excursions, which have returned to the northern zinc inevoutive
and perfectly made it disappeared. He could soon see the q

In [37]:
# Move models to evaluation mode
lstm_loaded.eval()
rnn_loaded.eval()
two_rnn_loaded.eval()

# Define how many samples you want
num_samples = 2

print("\n🔵 LSTM Generated Texts (Scale Factor 1.2):")
for i in range(num_samples):
    print(f"\nSample {i+1}:")
    print(sample(lstm_loaded, starting_str='The island', scale_factor=1.2))

print("\n🟢 RNN Generated Texts (Scale Factor 1.5):")
for i in range(num_samples):
    print(f"\nSample {i+1}:")
    print(sample(rnn_loaded, starting_str='The island', scale_factor=1.5))

print("\n🟣 Two-Layer RNN Generated Texts (Scale Factor 1.7):")
for i in range(num_samples):
    print(f"\nSample {i+1}:")
    print(sample(two_rnn_loaded, starting_str='The island', scale_factor=1.7))




🔵 LSTM Generated Texts (Scale Factor 1.2):

Sample 1:
The island was bottle, the Southerner
distance, he had only to wait himself in its turn. To the oars of the inexplicable face. The wool had broken so happies marched the channel, which was heard, he turned themselves a prison-cure, carried
all important change!” he cried, “what they would have been seen that in the morning. This lay to considerable suit to come to the scarcely the lava, the current late as the engineer and
it was more than unimportant from it.

“Yes, and if the balloon and Neb was to esca

Sample 2:
The island showed that the beach they would ascertain that a fibround seals. As American coast. The wall of the hummer that the trade still fired, and, that he had formed a single
numerous
breath, at a similar excursions, or it is your dimension made, for the most faithful day only some days to set out the south being knew to his country.

The convicts were near the species arriving on the island, the precision.
They di